### Definition Generation Evaluation

In [3]:
import json
# model = "llama"
model = "llama"

# Load the JSON data
with open(f'./dataset/{model}_definition_combined.json', 'r') as json_file:
    data = json.load(json_file)

#### Dataset creation

In [4]:
# Initialize lists to store predictions and references
predictions = []
references = []
references_bluert = []

# Iterate through each item in the JSON data
for item in data:
  celex_id = item["celex_id"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)

      references_bluert.append(original_text)

In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 7.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
  Using cached xxhash-3.4.1-cp39-cp39-macosx_11_0_arm64.whl (30 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 36.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 24.9 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl (174 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.8 MB/s eta 0:00:00
  Using cached async_ti

#### BLEU Score
- Reason for choosing
  
  - Length of the generated definitions matters
  - Precise definition is required


In [6]:
import evaluate
bleu = evaluate.load("bleu")

/Users/ashishchouhan/Desktop/PhD/6a.Production_Code/4. final_paper_code/LexDrafter/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

BLEU-4: 0.04665412304915669


In [8]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

BLEU-3: 0.07417244990220871


In [9]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

BLEU-2: 0.1307435422849298


In [10]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

BLEU-1: 0.250832269797197


#### BERTScore

In [11]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
import evaluate
bertscore = evaluate.load("bertscore")

In [13]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

In [ ]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

print(f"BERTScore_F1: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore F1: {mean_f1}")

In [ ]:
# Extract the precision scores
precision_scores = bertscore_results['precision']

# Calculate the mean of precision scores
mean_precision = sum(precision_scores) / len(precision_scores)

print(f"BERTScore_precision: {bertscore_results['precision']}")
print("--------")
print(f"Mean BERTScore precision: {mean_precision}")

In [ ]:
# Extract the recall scores
recall_scores = bertscore_results['recall']

# Calculate the mean of recall scores
mean_recall = sum(recall_scores) / len(recall_scores)

print(f"BERTScore_recall: {bertscore_results['recall']}")
print("--------")
print(f"Mean BERTScore recall: {mean_recall}")

#### BLEURT

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

In [ ]:
import evaluate
bleurt = evaluate.load("bleurt", 'BLEURT-20')

In [ ]:
# Compute the BLEURT score
bleurt_results = bleurt.compute(predictions=predictions, references=references_bluert)

In [ ]:
# Extract the scores
scores = bleurt_results['scores']

# Calculate the mean of F1 scores
mean_bleurt = sum(scores) / len(scores)

In [ ]:
print(f"BLEURT: {bleurt_results['scores']}")
print("--------")
print(f"Mean BLEURT: {mean_bleurt}")